A larger stationary example

In [1]:
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
import psoap
from psoap.data import lkca14
from psoap import matrix_functions
from psoap import covariance

from matplotlib.ticker import FormatStrFormatter as FSF
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator

In [2]:
lkca14.sort_by_SN()

In [3]:
order = 22
wl0 = 5165
wl1 = 5178

In [4]:
snind = 10
wl = lkca14.wl[snind,order,:]
# Sort by ind
ind = (wl > wl0) & (wl < wl1)

wl = wl[ind]
fl = lkca14.fl[snind,order,ind]
sigma = lkca14.sigma[snind,order,ind]
date1D = lkca14.date1D[snind]
n_epochs = 1
N = len(wl)
print("Matrices of size {}".format(N))

Matrices of size 289


In [20]:
a = 0.26
l = 5.98
n_draw = 5

In [21]:
V11 = covariance.get_C(wl, wl, a, l) 
V11[np.diag_indices_from(V11)] += 1e-10 * np.ones(N)

In [22]:
V11_c = np.empty((N, N), dtype=np.float64) 
matrix_functions.fill_V11_one(V11_c, wl, a, l)
V11_c[np.diag_indices_from(V11_c)] += 1e-10 * np.ones(N)

In [23]:
# draw_prior = np.random.multivariate_normal(np.ones(N), V11, size=n_draw)
draw_prior_short = np.random.multivariate_normal(np.ones(N), V11_c, size=n_draw)

In [24]:
l = 15.00
V11_c = np.empty((N, N), dtype=np.float64) 
matrix_functions.fill_V11_one(V11_c, wl, a, l)
V11_c[np.diag_indices_from(V11_c)] += 1e-10 * np.ones(N)

In [25]:
draw_prior_long = np.random.multivariate_normal(np.ones(N), V11_c, size=n_draw)

In [9]:
mu, Sigma = covariance.predict_one(wl, fl, sigma, wl, a, l)
Sigma[np.diag_indices_from(Sigma)] += 1e-10 * np.ones(N)

In [10]:
draw_post = np.random.multivariate_normal(mu, Sigma, size=n_draw)

In [26]:
ax_width = 3.0
ax_height = 1.0
ax_resid_height = 0.5

left_margin = 0.55
right_margin = 0.55

bottom_margin = 0.4
top_margin = 0.25

middle_margin = 0.1

y_min = 0.2
y_max = 1.6
dy = (y_max - y_min)/ax_height

delta_y_resid = dy * ax_resid_height / 2


xx = left_margin + right_margin + ax_width
yy = bottom_margin + top_margin + 3 * middle_margin + 3. * ax_height + ax_resid_height

fig = plt.figure(figsize=(xx,yy))

# in figure fraction
rect = [left_margin/xx, (yy - (top_margin + ax_height))/yy, ax_width/xx, ax_height/yy]

ax0 = fig.add_axes(rect)
for i in range(n_draw):
    ax0.plot(wl, draw_prior_long[i], lw=0.5)
ax0.xaxis.set_ticklabels([])
ax0.set_xlim(wl0, wl1)
ax0.set_ylim(y_min, y_max)

rect = [left_margin/xx, (yy - (top_margin + 2 * ax_height + middle_margin))/yy, ax_width/xx, ax_height/yy]

ax1 = fig.add_axes(rect)
for i in range(n_draw):
    ax1.plot(wl, draw_prior_short[i], lw=0.5)
ax1.xaxis.set_ticklabels([])
ax1.set_xlim(wl0, wl1)
ax1.set_ylim(y_min, y_max)



rect = [left_margin/xx, (yy - (top_margin + 3 * ax_height + 2*middle_margin))/yy, ax_width/xx, ax_height/yy]

ax2 = fig.add_axes(rect)
ax2.xaxis.set_ticklabels([])

ax2.errorbar(wl, fl, yerr=sigma, fmt="none", color="0.6", capsize=0.0, ecolor="0.6", lw=0.3)
ax2.plot(wl, fl, ".", color="0.4", ms=2)
for i in range(n_draw):
    ax2.plot(wl, draw_post[i], lw=0.5)
ax2.plot(wl, mu, "k", lw=0.7)

ax2.set_xlim(wl0, wl1)
ax2.set_ylim(y_min, y_max)



rect = [left_margin/xx, (yy - (top_margin + 3 * ax_height + 3 * middle_margin + ax_resid_height))/yy, 
        ax_width/xx, ax_resid_height/yy]
ax3 = fig.add_axes(rect)

ax3.errorbar(wl, fl - mu, yerr=sigma, fmt="none", color="0.6", capsize=0.0, ecolor="0.6", lw=0.3)
ax3.plot(wl, fl - mu, ".", color="0.4", ms=2)
ax3.set_xlim(wl0, wl1)
ax3.set_ylim(-delta_y_resid, delta_y_resid)
ax3.set_xlabel(r"$\lambda\;[\AA]$")

fig.savefig("single_star.pdf")
fig.savefig("single_star.png")

In [12]:
ans = covariance.optimize_GP_one(wl, fl, sigma, a, l)

In [13]:
ans

array([ 0.27434737,  6.24919004])